In [1]:
import logging
import re
from urllib.parse import urljoin,urlencode
import multiprocessing
import time
import json
from os import makedirs
from os.path import exists
import pandas as pd
from lxml import etree

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver import ChromeOptions

logging.basicConfig(level=logging.INFO,format='%(asctime)s - %(levelname)s: %(message)s')

# 基础网址和爬取页码数量设定
BASE_URL='https://www.9fzt.com/marketCenter/aStockMarket.html?tab=0'
TOT_PAGEs=4

In [2]:
def get_browser():
    option=ChromeOptions()
    option.add_experimental_option('excludeSwitches',['enable-automation'])
    option.add_experimental_option('useAutomationExtension',False)
    # 设置不显式地显示浏览器
    option.add_argument('--headless')
    browser=webdriver.Chrome(options=option)
    browser.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument",
                            {'source': 'Object.defineProperty(navigator,"webdriver",{get:()=>undefind})'})
    browser.implicitly_wait(10)
    return browser


In [3]:
def parse_page(page_html,protol_type='https:'):
    pattern=re.compile('序号(.*)',re.S)
    stock_urls=re.findall(pattern,page_html)
    pattern=re.compile('<a href="(.*?)".*? class="bluelink ff_din-medium fw-500" target="_blank" rel="noopener">',re.S)
    stock_urls=re.findall(pattern,str(stock_urls))
    stock_urls=[urljoin(protol_type,url) for url in stock_urls]
    return stock_urls


In [4]:
def scrape_stock_list(browser,page_num):
    browser.get(BASE_URL)
    urls=[]
    WebDriverWait(browser,20,0.5).until(lambda browser:len(
        browser.find_element(By.XPATH,'//*[@id="__next"]/div/div[3]/div[2]/ul[20]/li[13]/span').text)>0)
    for page in range(page_num):
        urls+=(parse_page(browser.page_source))
        time.sleep(0.1)
        ac=ActionChains(browser)
        # 鼠标移动到下一页按钮上
        ac.move_to_element(browser.find_element_by_name('whj_nextPage')).perform()
        # 点击确定跳转至下一页
        ac.click(browser.find_element_by_name('whj_nextPage')).perform()
        time.sleep(0.1)
        WebDriverWait(browser,10,0.5).until(lambda browser: len(
            browser.find_element(By.XPATH,'//*[@id="__next"]/div/div[3]/div[2]/ul[20]/li[13]/span').text)>0)
    print(urls)
    return urls


In [5]:
def scrape_stockprice_page(browser,url):
    logging.info('scraping %s...',url)
    browser.get(url)
    WebDriverWait(browser,60,2).until(lambda browser: browser.find_element_by_id('stockprice').text != '--')
    return browser.page_source


In [6]:
def scrape_company_list(browser,urls):
    companys=[]
    for url in urls:
        logging.info('scraping %s...',url)
        browser.get(url)
        logging.info('scraping 最新动态...')
        browser.find_element_by_link_text('最新动态').click()
        browser.switch_to.window(browser.window_handles[1])
        WebDriverWait(browser,10,2).until(lambda browser: len(
           browser.find_element(By.XPATH,'//*[@id="gsgg"]/div[2]/div/div/section/div/div/div/div/div/div/table/thead/tr/th[2]/span/div/span[1]').text)>0)
        shareholder=browser.page_source
        browser.close()
        browser.switch_to.window(browser.window_handles[0])

        companys.append([shareholder])
    return companys


In [7]:
# 使用xpath匹配
date_xpath='//*[@id="gsgg"]/div[2]/div/div/section/div/div/div/div/div/div/table/tbody/tr[1]/td[1]/text()'
title_xpath='//*[@id="ggTitleNoti0"]/text()'

def parse_company_data(html):
    html=etree.HTML(html)

    date=html.xpath(date_xpath)[0]
    title=html.xpath(title_xpath)[0]

    return [date,title]


In [8]:
def get_table_company_data(data):
    table=pd.DataFrame(columns=['date','title'])
    for item in data:
        table.loc[len(table)]=parse_company_data(item)
    return table


In [9]:
browser=get_browser()
urls=scrape_stock_list(browser,TOT_PAGEs)
#urls=['https://stock.9fzt.com/index/bj_870204.html']

companys=scrape_company_list(browser,urls)
data=[company[0] for company in companys]

company_data_table=get_table_company_data(data)

company_data_table.to_csv('news_data_table.csv',index=False,encoding='gbk')


2023-07-04 19:24:28,442 - INFO: scraping https://stock.9fzt.com/index/sz_301488.html...


['https://stock.9fzt.com/index/sz_301488.html', 'https://stock.9fzt.com/index/bj_833533.html', 'https://stock.9fzt.com/index/sz_301007.html', 'https://stock.9fzt.com/index/sz_301255.html', 'https://stock.9fzt.com/index/sz_300489.html', 'https://stock.9fzt.com/index/sz_301221.html', 'https://stock.9fzt.com/index/sz_300552.html', 'https://stock.9fzt.com/index/sh_688280.html', 'https://stock.9fzt.com/index/sz_300503.html', 'https://stock.9fzt.com/index/sz_300549.html', 'https://stock.9fzt.com/index/sh_688331.html', 'https://stock.9fzt.com/index/bj_831278.html', 'https://stock.9fzt.com/index/sh_688147.html', 'https://stock.9fzt.com/index/sh_688123.html', 'https://stock.9fzt.com/index/bj_836221.html', 'https://stock.9fzt.com/index/sh_688326.html', 'https://stock.9fzt.com/index/sz_301099.html', 'https://stock.9fzt.com/index/sh_688071.html', 'https://stock.9fzt.com/index/sz_300496.html', 'https://stock.9fzt.com/index/sz_301183.html', 'https://stock.9fzt.com/index/sz_301023.html', 'https://sto

2023-07-04 19:24:33,102 - INFO: scraping 最新动态...
2023-07-04 19:24:35,956 - INFO: scraping https://stock.9fzt.com/index/bj_833533.html...
2023-07-04 19:24:37,887 - INFO: scraping 最新动态...
2023-07-04 19:24:43,717 - INFO: scraping https://stock.9fzt.com/index/sz_301007.html...
2023-07-04 19:24:45,068 - INFO: scraping 最新动态...
2023-07-04 19:24:49,668 - INFO: scraping https://stock.9fzt.com/index/sz_301255.html...
2023-07-04 19:24:51,251 - INFO: scraping 最新动态...
2023-07-04 19:24:56,640 - INFO: scraping https://stock.9fzt.com/index/sz_300489.html...
2023-07-04 19:24:59,175 - INFO: scraping 最新动态...
2023-07-04 19:25:04,073 - INFO: scraping https://stock.9fzt.com/index/sz_301221.html...
2023-07-04 19:25:05,393 - INFO: scraping 最新动态...
2023-07-04 19:25:10,381 - INFO: scraping https://stock.9fzt.com/index/sz_300552.html...
2023-07-04 19:25:13,018 - INFO: scraping 最新动态...
2023-07-04 19:25:17,710 - INFO: scraping https://stock.9fzt.com/index/sh_688280.html...
2023-07-04 19:25:20,143 - INFO: scraping 

2023-07-04 19:29:36,444 - INFO: scraping 最新动态...
2023-07-04 19:29:39,078 - INFO: scraping https://stock.9fzt.com/index/sz_000020.html...
2023-07-04 19:29:40,493 - INFO: scraping 最新动态...
2023-07-04 19:29:43,278 - INFO: scraping https://stock.9fzt.com/index/sh_603015.html...
2023-07-04 19:29:44,639 - INFO: scraping 最新动态...
2023-07-04 19:29:47,468 - INFO: scraping https://stock.9fzt.com/index/sh_603045.html...
2023-07-04 19:29:48,870 - INFO: scraping 最新动态...
2023-07-04 19:29:51,924 - INFO: scraping https://stock.9fzt.com/index/sh_603085.html...
2023-07-04 19:29:53,468 - INFO: scraping 最新动态...
2023-07-04 19:29:56,784 - INFO: scraping https://stock.9fzt.com/index/sz_002403.html...
2023-07-04 19:29:58,255 - INFO: scraping 最新动态...
2023-07-04 19:30:01,455 - INFO: scraping https://stock.9fzt.com/index/sz_301052.html...
2023-07-04 19:30:02,904 - INFO: scraping 最新动态...
2023-07-04 19:30:05,598 - INFO: scraping https://stock.9fzt.com/index/sz_002976.html...
2023-07-04 19:30:07,094 - INFO: scraping 